In [46]:
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
crf=21.0
qcomp=0.6
aq_mode=3
aq_s=0.6
newline="\r\n"

PROCUCT_DEST_DIR=r"C:\Users\heber\Music"

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="3077739"

SRC="A Prophet AKA Un Prophete 2009 1080p Blu-ray AVC DTS-HD MA 5.1-DIY@PTHome".strip().replace(" ",".")
print(SRC)
REMUX_DIRNAME="A.Prophet.AKA.Un.Prophete.2009.1080p.Bluray.AVC.Remux.DTS-HD.MA.5.1-PTH"
ENCODE_FILENAME="A.Prophet.AKA.Un.Prophete.2009.1080p.Bluray.REMUX.DTS-PTH.mkv"
MOVIE=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

ACKNOWLEDGEMENT='''原盘来自@无后缀，字幕取自CMCT。
菜单未修改，追加简 繁双字幕，右键调出。
'''

SOUNDTRACK="【法语】【简|繁|法字幕】"
# SOUNDTRACK="【英语】【简|繁|简英|繁英|英字幕】"

SUBTITLE_GZ=1 #官字组

LOG_DIR=os.getcwd()

A.Prophet.AKA.Un.Prophete.2009.1080p.Blu-ray.AVC.DTS-HD.MA.5.1-DIY@PTHome


In [34]:
metainfo=getmetainfo(DOUBAN_ID)
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()

In [47]:
# crop=subprocess.getoutput("ffmpeg -i "+MOVIE+" -t 1 -vf cropdetect -f null - 2>&1")
#ffmpeg -i $MOVIE -t 1 -vf cropdetect -f null - 2>&1 | awk '/crop/ { print $NF }' | tail -1
# try: crop="-filter:v \""+re.findall(r'crop=.*',crop)[-1]+"\""
# except: pass
# MOVIE=r"/sdcard/Freaky.2020.1080p.BluRay.REMUX.DTS-louco@PTH.mkv"

#deinterlace: yadif
#denoise: hqdn3d
# crop: crop=in_w-480:in_h,

crop=""
# crop='-filter:v "hqdn3d=12.0"'
# crop='-filter:v "crop=in_w:in_h-280"'
crop='-filter:v "crop=in_w:in_h-42"'
test=""

#bt2020
# hdrmeta='--colorprim 9 --colormatrix 9 --transfer 16 --hdr10 --max-cll "800,279" \
# --master-display G(8500,39850)B(6550,2300)R(35400,14600)WP(15635,16450)L(40000000,20)'
#p3
# hdrmeta='--colorprim 9 --colormatrix 9 --transfer 16 --hdr10 \
# --master-display G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(10000000,1)'
hdrmeta=""

#--demuxer y4m --preset slower --profile high --level 4.1
x264_stmt=f"x264  --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

# --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower
# --pools 12
x265_stmt=f"x265 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --pools 36 --numa 48 \
--rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 {hdrmeta} --output 265.mkv -"

# print(x265_stmt)
# test="-ss 3700 -t 60"

_args=x264_stmt.replace(" --","\n --").split("\n")[1:-1]
args=[]
for i in _args:
    newitem=i.strip().replace("--","").replace(" ","=").replace(":",",")
    if "=" not in newitem:
        newitem+="=1"
    args.append(newitem)
# args=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args]

_args5=x265_stmt.replace(" --","\n --").split("\n")[1:-1]
args5=[]
for i in _args5:
    newitem=i.strip().replace("--","").replace(" ","=").replace(":",",")
    if "=" not in newitem:
        newitem+="=1"
    args5.append(newitem)
        
# args5=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args5]

# x264_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x264")
x264_filename=ENCODE_FILENAME.replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
# x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_filename=ENCODE_FILENAME.replace("REMUX", "x265.10bit")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

if not os.path.exists(os.path.join(PROCUCT_DEST_DIR, x264_dirname)):
    os.mkdir(os.path.join(PROCUCT_DEST_DIR, x264_dirname))
if not os.path.exists(os.path.join(PROCUCT_DEST_DIR, x265_dirname)):
    os.mkdir(os.path.join(PROCUCT_DEST_DIR, x265_dirname))

torrent264=f"transmission-create -p -o {os.path.join(PROCUCT_DEST_DIR,x264_dirname)}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x264_dirname)}"
torrent265=f"transmission-create -p -o {os.path.join(PROCUCT_DEST_DIR,x265_dirname)}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x265_dirname)}"

cmd264=""
cmd265=""

if "\\" in os.getcwd():
    cmd264="del x264.log\n"
    cmd265="del x265.log\n"

if "DTS" in REMUX_DIRNAME:
    cmd264+=f"ffmpeg -y -c:a dca -core_only true {test} -i {MOVIE} -c:v libx264 \
-profile:v high {crop} -preset slower -x264-params {':'.join(args)} \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V  -report {x264_fullpath} 2>&1 | tee x264.log \n\
mkvpropedit {x264_fullpath} --add-track-statistics-tags\n"

    cmd265+=f"ffmpeg -y -c:a dca -core_only true {test} -i {MOVIE} \
-pix_fmt yuv420p10le -c:v libx265 -profile:v main10 {crop} -preset slower -x265-params {':'.join(args5)} \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V  -report {x265_fullpath} 2>&1 | tee x265.log \n\
mkvpropedit {x265_fullpath} --add-track-statistics-tags\n"
else:
    if "LPCM" in REMUX_DIRNAME:
        acodec="-c:a flac"
    else:
        acodec="-bsf:a dca_core -c:a dts"
    cmd264+=f"ffmpeg -y {test} -i {MOVIE} -c:v libx264 \
-profile:v high {crop} -preset slower -x264-params '{':'.join(args)}' \
-max_muxing_queue_size 1024 {acodec} -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V -strict -2 -report {x264_fullpath} 2>&1 | tee x264.log \n\
mkvpropedit {x264_fullpath} --add-track-statistics-tags\n"

    cmd265+=f"ffmpeg -y {test} -i {MOVIE} \
-pix_fmt yuv420p10le -c:v libx265 -profile:v main10 {crop} -preset slower -x265-params '{':'.join(args5)}' \
-max_muxing_queue_size 1024 {acodec}  -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V -strict -2 -report {x265_fullpath} 2>&1 | tee x265.log \n\
mkvpropedit {x265_fullpath} --add-track-statistics-tags\n"
    
print(len(test))
if len(test)==0:
    cmd264_2=get_captures("x264",x264_fullpath)+"\n"+torrent264+"\n"
    cmd265_2=get_captures("x265",x265_fullpath)+"\n"+torrent265+"\n"
else:
    cmd264_2=""
    cmd265_2=""
    print(cmd265_2)


    
remux_fullpath=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")
cmdremux=get_captures("REMUX",remux_fullpath)
cmdremux+=f"transmission-create -p -o {os.path.join(PROCUCT_DEST_DIR,REMUX_DIRNAME)}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME)}"

# with open("x264.sh","w") as f:
#     f.write(f"export FFREPORT=file=x264.log:level=32\n{cmd264}\nunset FFREPORT\n{cmd264_2}")
# with open("x265.sh","w") as f:
#     f.write(f"export FFREPORT=file=x265.log:level=32\n{cmd265}\nunset FFREPORT\n{cmd265_2}")
# with open("x264.bat","w") as f:
#     f.write(f"SET FFREPORT=file=x264.log:level=32\n{cmd264}\nSET \"FFREPORT=\"\n{cmd264_2}")
# with open("x265.bat","w") as f:
#     f.write(f"SET FFREPORT=file=x265.log:level=32\n{cmd265}\nSET \"FFREPORT=\"\n{cmd265_2}")    

with open("x264.sh","w") as f:
    f.write(f"{cmd264}\n{cmd264_2}")
with open("x265.sh","w") as f:
    f.write(f"{cmd265}\n{cmd265_2}")
with open("x264.bat","w") as f:
    f.write(f"{cmd264}\n{cmd264_2}")
with open("x265.bat","w") as f:
    f.write(f"{cmd265}\n{cmd265_2}") 
    
with open("remux.sh","w") as f:
    f.write(cmdremux)
with open("remux.bat","w") as f:
    f.write(cmdremux)
    
print('''Execute:
bash x264.sh && bash x265.sh && bash remux.sh

x264.bat && x265.bat && remux.bat
''')

#ffmpeg -i dest.mkv -i remux.mkv -map 0:v -map 0:s -map 1:a -c:v copy -c:a dts -strict -2 dtsout.mkv


0
Execute:
bash x264.sh && bash x265.sh && bash remux.sh

x264.bat && x265.bat && remux.bat



In [48]:
encode_picbed=upload_folder("x265_captures",smms)

===UPLOAD===
https://i.loli.net/2021/04/20/HRlj8oLiUZI3qfs.jpg
https://i.loli.net/2021/04/20/63R78M5CmzQNs4t.jpg
https://i.loli.net/2021/04/20/EDInjs9eVBz2dCk.jpg
https://i.loli.net/2021/04/20/ZKqvWpBLRAxtkUe.jpg
https://i.loli.net/2021/04/20/6CH3AKvzOQ2lNiP.jpg
https://i.loli.net/2021/04/20/vRs4kO7V5hgTCNQ.jpg


In [49]:
remux_picbed=upload_folder("REMUX_captures",smms)

===UPLOAD===
https://i.loli.net/2021/04/20/n9E2QkXeRMdtjO7.jpg
https://i.loli.net/2021/04/20/L7u3PZhaXzJ9Tx8.jpg
https://i.loli.net/2021/04/20/vO7i3aL2kCcgGum.jpg
https://i.loli.net/2021/04/20/2DcHZEW3jeLlNOg.jpg
https://i.loli.net/2021/04/20/EC5g34nXKcNdYVx.jpg
https://i.loli.net/2021/04/20/oh7cg6fGrCAslj5.jpg


In [26]:
# del get_bbcode
with open(f"x264.bbcode.txt","w") as f:
    f.write(get_bbcode(x264_fullpath, src=SRC, logname="x264.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.txt","w") as f:
    f.write(get_bbcode(x265_fullpath, src=SRC, logname="x265.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.txt","w") as f:
    f.write(get_bbcode(remux_fullpath, src=SRC, picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))

Title: C:\Users\heber\Music\As Far As My Feet Will Carry Me 2001 1080p BluRay x264 DTS-PTH\As Far As My Feet Will Carry Me 2001 1080p BluRay x264 DTS-PTH mkv


UnicodeEncodeError: 'gbk' codec can't encode character '\xdf' in position 287: illegal multibyte sequence

In [51]:
with open(f"x264.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x264_fullpath, src=SRC, logname="x264.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x265_fullpath, src=SRC, logname="x265.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.base64.txt","w") as f:
    f.write(getbase64json(remux_fullpath, src=SRC, logname="",\
                          picbed=remux_picbed, metainfo=metainfo, is_remux=True, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))

Title: C:\Users\heber\Music\A Prophet AKA Un Prophete 2009 1080p Bluray x264 DTS-PTH\A Prophet AKA Un Prophete 2009 1080p Bluray x264 DTS-PTH mkv
Title: C:\Users\heber\Music\A Prophet AKA Un Prophete 2009 1080p Bluray x265 10bit DTS-PTH\A Prophet AKA Un Prophete 2009 1080p Bluray x265 10bit DTS-PTH mkv
Title: C:\Users\heber\Music\A Prophet AKA Un Prophete 2009 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH\A Prophet AKA Un Prophete 2009 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH mkv
